In [ ]:
import os
# Specify the new directory path
parent_directory = 'D:/Projects/CornYield/src'

# Change the current directory
os.chdir(parent_directory) 
# %pwd


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

import torch
import torch.optim as optim

from dataset import select_data_and_yield_list
from ml.ml_predict import plot_result_separate
from dl.dl_dataset import CornDataset
from dl.model import ResNetRegression, ResNetFeatures, NNRegression
from dl.train import train_with_cross_validation,  validate_dual_models
from dl.dl_prediction import data_prepare

In [ ]:
from dataset import get_filename_metadata
from dl.dl_dataset import MixedDataset
from dl.train import train_with_cross_validation, train, validate, data_transform

In [ ]:
yield_file = 'D:/Corn_Yield/BL2022_Yld.csv'

img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220705_DOY186_extracted_filled'
out_path = '../output/DOY186/'

is_save_model = True

In [ ]:
import random

In [ ]:
key_word_list = ['Ref_filled.tif']
analyze_variety_id = 0
for keyword in key_word_list:

    img_list, yield_pf = get_filename_metadata(img_path, yield_file, keyword)

    
    indices = yield_pf.index[yield_pf['Variety_int'] == analyze_variety_id].tolist()
    img_list = [img_list[i] for i in indices]
    yield_pf = yield_pf[yield_pf['Variety_int'] == analyze_variety_id]
    yield_pf = yield_pf.reset_index(drop=True)
    yield_list = list(yield_pf['Yield_Bu_Ac'])
    
    print(len(img_list))
    print(len(yield_pf))
    
    random_numbers = [random.randint(0, len(img_list)) for _ in range(100)]
    for i in random_numbers:
        # print(yield_pf.iloc[i, 1])
        # print(img_list[i][77:89])
        print(img_list[i][77:89] == yield_pf.iloc[i, 1])

 

In [ ]:
    
selection = ['Pioneer'] # 
# selection = 'Pioneer Deficit' 
# selection = 'Pioneer Full'

# key_word_list = ['Ref_filled.tif', 'RGB_filled.tif']
key_word_list = ['Ref_filled.tif']
suffix_list_list = [[], ['LWIR_filled.tif']]

# suffix = ['base', 'lwir']
# suffix_list = ['LWIR_filled.tif']
# VI_list = ['ndvi', 'ndre', 'gndvi', 'evi']
VI_list = ['evi']
for keyword in key_word_list:
    pioneer_yield_list, train_val_dataset, test_dataset, irrigate_type_list, test_indices= data_prepare(yield_file, img_path, out_path, keyword, selection)
    # yield_file = 'D:/Corn_Yield/BL2022_Yld.csv'
    doy_name = img_path[-23:-17]

            
    in_channel = 5
    num_epochs = 200
    batch_size = 32

    # Initialize an empty list to store fold-wise performance
    fold_accuracies = []

    # Initialize a new model for each fold
    # model = ResNetRegression(in_channel)
    model = ResNetFeatures(in_channel, 512)
    model2 = NNRegression(512)
    # model = EncoderCNN(in_channel, 1)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    model.to(device)
    model2.to(device)


    criterion = torch.nn.MSELoss()  # Mean Squared Error loss function
    # optimizer = optim.Adam(list(conv.parameters()) + list(deconv.parameters()), lr=0.001)  # Adam optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

    # optimizer.zero_grad()
    model = train_with_cross_validation(model, train_val_dataset, batch_size, num_epochs, optimizer, criterion, dual_model=True, model2=model2)

    if is_save_model:
        model_name = "path/model_" + model.__class__.__name__+ doy_name +"_Batch=" +str(batch_size) + "_state_dual.pth"
        torch.save(model.state_dict(), model_name)

    test_accuracy, test_prediction = validate_dual_models(model, model2, test_dataset, criterion, batch_size = batch_size, is_return_output = True)

    print(f'validation mse is {np.sqrt(np.mean(test_accuracy))}')


    name_tag = doy_name
    out_name = name_tag + ' ' + 'Pioneer' + ' ' 
    out_name = out_name + keyword[:-11] 
    out_name = out_name + ' CNN '

    yield_data = np.array(pioneer_yield_list)
    irrigate_data = np.array(irrigate_type_list)

    test_truth = yield_data[test_indices]
    plot_result_separate(np.array(test_truth), np.array(test_prediction), test_indices, irrigate_data, out_name)


In [ ]:
model2 = ResNetRegression(5,1)
print(model2)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import numpy as np
import pandas as pd
import os


crop_var={
    3: 'all',
    2: 'Pioneer',
    1: 'CH 192-10',
    0: 'DKC 51-91'
}

irrigate_var={
    2: 'All',
    0: 'Full',
    1: 'Deficit'
}


def plot_distinct_yields(y_test, y_pred, irrigate_data, variety_data, title, save_name, value_range = [4,19]  ):
    
     
    accurate_metric = []
    
    x = np.arange(value_range[0], value_range[1]+1)
    plt.plot(x, x, color = 'k', ls='--', alpha=0.7, linewidth=1.25)
    
    color_list=['#2ca02c', '#bcbd22', 'r']
    marker_list = ['d', 'o', '+']
    # marker_list = ['o', '+','d']
    
    for j in np.sort(np.unique(variety_data)):
        marker=marker_list[j]
        for i in np.sort(np.unique(irrigate_data)):
            marker=marker_list[2-i]
            color = color_list[i]
        # marker=marker_list[i]
            # color = color_list[j]
            label = irrigate_var[i] + ' ' + crop_var[j] 

            condition = (irrigate_data == i) & (variety_data == j)

            if marker != '+':
                plt.scatter( y_test[condition], 
                        y_pred[condition],
                        marker=marker, s=25, color = color, label = label, alpha=0.5, edgecolors=color)
            else:
                plt.scatter( y_test[condition], 
                        y_pred[condition],
                        marker=marker, s=25, color = color, label = label)
            
            rmse=np.sqrt(mean_squared_error(y_test[condition], y_pred[condition]))
            mae=mean_absolute_error(y_test[condition], y_pred[condition])
            r2=r2_score(y_test[condition], y_pred[condition])

            accurate_metric.append([label, rmse, mae, r2, np.mean(y_test), 100*rmse/np.mean(y_test), 100*mae/np.mean(y_test)])

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mare = np.mean(np.abs((y_test - y_pred)/y_test))
    accurate_metric.append(['all', rmse, mae, r2, np.mean(y_test), 100*rmse/np.mean(y_test), 100*mae/np.mean(y_test)])
    
    plt.xlim(value_range)
    plt.ylim(value_range)
    plt.xticks(fontname='Times New Roman', size=10)
    plt.yticks(fontname='Times New Roman', size=10)
    plt.xlabel('Ground Measured Yield (ton/ha)', fontname = 'Times New Roman', fontsize = 12)
    plt.ylabel('Predicted Yield (ton/ha)', fontname = 'Times New Roman', fontsize = 12)

    plt.title(title, fontname = 'Times New Roman', fontsize = 14)

    plt.text(value_range[0]*1.1, value_range[1]*0.96, f'R-squared = {round(r2,2)}', 
             fontname = 'Times New Roman', fontsize = 11)
    plt.text(value_range[0]*1.1, value_range[1]*0.92, f'RMSE = {round(rmse, 2)}',  
             fontname = 'Times New Roman', fontsize = 11)
    # plt.text(4500, 17300, f'MAE = {round(mae, 2)} (Kg/Ha)',  
    #          fontname = 'Times New Roman', fontsize = 11)
    plt.text(value_range[0]*1.1, value_range[1]*0.88, f'MARE = {round(mare, 2)}',  
             fontname = 'Times New Roman', fontsize = 11)
    
    font_props = FontProperties(family='Times New Roman', size=11)
    plt.legend(loc='lower right', frameon=False, 
               prop=font_props)
    # yy_pred = model_lasso.predict(X_train)
    # plt.scatter(y_train, yy_pred)
    plt.savefig(save_name + '.png', dpi=300, bbox_inches='tight')
    # plt.show()
    plt.close()
    
    
    df = pd.DataFrame(accurate_metric, columns=['Name', 'RMSE', 'MAE', 'R2', 'mean_truth', 'rmse/mean', 'mae/mean'])
    csv_file_path = save_name+'_metric.csv'
    # Save DataFrame to CSV
    df.to_csv(csv_file_path, index=False)

    print(save_name, f' r-squared = {r2}, rmse = {rmse}, mae={mae}', 100*mae/np.mean(y_test))

''' Function to plot figure from data'''
def plot_from_csv(csv_file):

    base_filename = os.path.basename(csv_file)
    dir_file = os.path.dirname(csv_file)
    words = base_filename.split()
    name_tag = words[0] + ' '
    title_part2 = ' '
    if '+' in words[2]:
        pos = words[2].find('+')
        temp = 'Ref'if 'ref' in words[2].lower() else 'RGB'
        title_part2 += temp + words[2][pos:]
        # datasource = words[2][:pos].capitalize() if 'ref' in words[2].lower else 'RGB'
        # title_part2 += words[2][:pos].capitalize()  + words[2][pos:]
    else:
        words[2] = 'Ref'if 'ref' in words[2].lower() else 'RGB'
        title_part2 +=  words[2]

    title_part2 += ' ('+ words[3] + ')'
    

    df = pd.read_csv(csv_file)
    test_yields = df['Truth'].values
    pred_yields = df['Prediction'].values

    test_irrigate_data = df['Irrigation_int'].values
    test_variety_data = df['Vriaty_int'].values

    
    test_yields = np.array(test_yields)
    pred_yields = np.array(pred_yields)
    for var_id in set(test_variety_data):
        sub_test_yields = test_yields[test_variety_data == var_id]
        sub_pred_yields = pred_yields[test_variety_data == var_id]
        sub_test_irrigate_data = test_irrigate_data[test_variety_data == var_id]
        sub_test_variety_data = test_variety_data[test_variety_data == var_id]
        subtitle = name_tag + crop_var[var_id] + title_part2
        plot_distinct_yields(sub_test_yields*0.06277, sub_pred_yields*0.06277, 
                                sub_test_irrigate_data, sub_test_variety_data, subtitle, os.path.join(dir_file,subtitle))




In [ ]:


# path = 'C:\\Zhou\\OneDrive\\OneDrive - Oklahoma A and M System\\output\\' 

plot_from_csv("C:\\Zhou\\Ma\\Thesis\\PAPER\\Resnet18\\Resnet18_120\\DOY186 Pioneer Ref resnet18 .csv")